In [24]:
import pandas as pd
from pulp import LpMinimize, LpVariable, LpProblem, lpSum 
import numpy as np
from scipy.optimize import linear_sum_assignment

print("Work of Rybchynchuk Vladyslav AP-31")
print("Lab 8")

Work of Rybchynchuk Vladyslav AP-31
Lab 8


Автоматизація процесу розв'язку задачі 6 методом північно-західного кута.

In [29]:
def northwest_corner_method(supply, demand, costs): 
    rows, cols = len(supply), len(demand) 
    allocation = np.zeros((rows, cols)) 
 
    i, j = 0, 0 
 
    while i < rows and j < cols: 
        quantity = min(supply[i], demand[j]) 
        allocation[i, j] = quantity 
        supply[i] -= quantity 
        demand[j] -= quantity 
 
        if supply[i] == 0: 
            i += 1 
        if demand[j] == 0: 
            j += 1 
 
    return allocation 

def calculate_total_cost(allocation, costs):
    total_cost = np.sum(allocation * costs)
    return total_cost

# Початкові дані 
supply = [200, 250, 200, 130] 
demand = [190, 100, 120, 110, 130] 
costs = np.array([ 
    [28, 27, 18, 27, 24], 
    [18, 26, 27, 32, 21], 
    [27, 33, 23, 31, 34], 
    [190, 100, 120, 110, 130] 
]) 

# Виклик функції та виведення результатів
allocation = northwest_corner_method(supply, demand, costs)

print("Оптимальний розподіл:")
display(pd.DataFrame(allocation))
print("Загальна вартість оптимального розподілу:", calculate_total_cost(allocation, costs))

Оптимальний розподіл:


,0,1,2,3,4
0,190.0,10.0,0.0,0.0,0.0
1,0.0,90.0,120.0,40.0,0.0
2,0.0,0.0,0.0,70.0,130.0
3,0.0,0.0,0.0,0.0,0.0


Загальна вартість оптимального розподілу: 19040.0


Автоматизація процесу розв'язку задачі 1 методом потенціалів.

In [25]:
supply = [200, 450, 250]
suppliers = range(len(supply))

demand = [100, 125, 325, 250, 100]
consumers = range(len(demand))

costs = [[5, 8, 7, 10, 3], 
         [4, 2, 2, 5, 6],
         [7, 3, 5, 9, 2]]

costs = makeDict ([suppliers, consumers], costs, 0)

prob = LpProblem("Transportation_Problem", LpMinimize)

routes = [(i, j) for i in suppliers for j in consumers]

vars = LpVariable.dicts("Route", (suppliers, consumers), 0, None, LpInteger)

prob += (
    lpSum([vars[w][b] * costs[w][b] for (w, b) in routes]),
    "Sum_of_Transporting_Costs",
)

for s in suppliers:
    prob += (
        lpSum([vars[s][b] for b in consumers]) <=supply[s],
        "sum_of_Products_out_of_suppliers_%s" % s,
    )

for c in consumers:
    prob += (
        lpSum([vars[w][c] for w in suppliers]) >=demand[c],
        "Sum_of_Products_into_consumer%s" % c,
    )

prob.solve()

results = [v.varValue for v in prob.variables()]

print("Відповідь: ")
df = pd.DataFrame(allocation.reshape(4, 5)) 
# змінюємо індексацію 
df.index = np.arange(1, df.shape[0] + 1) 
df.columns = np.arange(1, df.shape[1] + 1) 
display(df)
print("Загальна вартість:", value(prob.objective))

Відповідь: 


,1,2,3,4,5
1,140.0,90.0,0.0,0.0,0.0
2,0.0,0.0,160.0,90.0,0.0
3,0.0,0.0,0.0,20.0,150.0
4,0.0,0.0,0.0,0.0,0.0


Загальна вартість: 3450.0
